# Generating fake data for neo4j analysis 

## Tasks 

* Define relevant node types
    - tilt 
        - meta  
        - dataDisclosed
            - Recipient
            - purposes
* Generate nodes 
    * Get list of company names
    * Define likelihood of same connections (sample size / no. of companies)
* Construct mutation (possibly combine with previous step)
* Figure out how to upload to neo4j as separate graph
* Do analysis

In [7]:
company_names = ['']


def build_mutation(company_names):

    """
    func:   build_mutations
            Generates a mutation string to pass to graphQL API that generates a node
    """


    # initialize
    output_string = 'mutation { createtilts( input: [ { '
    
    # add tilt and meta data
    output_string += '{{ createtilts( input: [ {{ _id:{{create:[{{oid: %s}}]}}meta:{create:[{{_id: {}, name: {}, created: {}, modified: "None", version: "1", language: "en", status: "active", url: {}, _hash: }}]}}' 
    
    # complete string
    output_string += '}] ) {tilts {meta { name } } }}'

    return output_string

In [110]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep


def get_table(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    elems = []
    for child in soup.find_all('table')[0].children:

        for some in child:
            row = []
            for i,td in enumerate(some):
                try:
                    row.append(td.text.replace('\n',""))
                except:
                    continue
                if len(row) > 0:
                    elems.append(row)

    return elems
    

table = get_table("https://www.zyxware.com/articles/4344/list-of-fortune-500-companies-and-their-websites")


In [122]:
subset = [[table[i][1],table[i][3], table[i][5]]   for i in range(0,len(table), 7)]

In [127]:
df = pd.DataFrame(subset[1:], columns=subset[0])

In [128]:
df

,Rank,Company,Website
0,1,Walmart,https://www.walmart.com/
1,2,Amazon,https://www.amazon.com/
2,3,Apple,https://www.apple.com/
3,4,CVS Health,https://www.cvshealth.com/
4,5,UnitedHealth Group,https://www.unitedhealthgroup.com/
...,...,...,...
495,496,Hasbro,https://shop.hasbro.com/
496,497,Roper Technologies,https://www.ropertech.com/
497,498,ARKO,https://www.arkocorp.com/
498,499,Frontier Communications,https://frontier.com/
